**This notebook contains a script that combines all the SETA POE documents into one submission Folder**

**Add Local Library to Path**

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [3]:
import pandas as pd
import re
from local_library import list_files
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Temp Library**

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#authenticate Google
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)


def move_file(old_folder_id, new_folder_id, exclusion_array=[]):
    """
    Moves a google drive files from one folder to another.
    
    Args:
        old_folder_id: A string representing the old folder google drive id.
        new_folder_id: A string representing the new folder google drive id.
        exclusion_array: An array of strings containing the titles to be excluded from the copy.
        
    Returns:
        _: A boolean representing success or failure
    """
    
    files = list_files(old_folder_id)
    for file in files:
        if file['title'] not in exclusion_array:
            copy_file(file['id'], file['title'], new_folder_id)
    return True
    
def copy_file(file_id, file_title, new_folder_id):
    """
    Creates copy of file in destination folder
    
    Args:
        file_id: I.D of the file to be copied
        file_title: Name of the file to be copied
        new_folder_id: Folder where copy will be stored.
    Return:
        none
    """
    
    copied_file = {"title": file_title,
                    'parents': [{
            'kind': 'drive#fileLink',
            'id': new_folder_id
        }]}

    drive.auth.service.files().copy(fileId=file_id,
                                    body=copied_file,
                                    supportsAllDrives=True
    ).execute()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Map Documents folders**

In [5]:
files = list_files("1W04tIGaWpqSgKsaP6xSjrUtP1OsVcaPX")

In [6]:
student_documents_folders = []
for file in files:
    student_documents_folders.append([file['title'], file['id']])


In [7]:
documents_df = pd.DataFrame(student_documents_folders, columns=["Name", "Documents ID"])

**Map Evidence Folders**

In [8]:
files_2 = list_files("1-2tkFz7qrd2MD-vvudm0AlyK46YFt5so")

In [9]:
student_evidence_folders = []
for file in files_2:
    student_evidence_folders.append([file['title'], file['id']])

In [10]:
evidence_df = pd.DataFrame(student_evidence_folders, columns=["Name", "Evidence ID"])

**Merge Documents and Evidence Folders**

In [11]:
folders_df = pd.merge(documents_df, evidence_df, how="inner")

In [12]:
#folders_df = folders_df[folders_df['Name'] == 'Adam Saul David Becker']
#folders_df.head()

**Import and Merge Student Data**

In [13]:
#import data
cohort_df = import_worksheet('2020 SETA Submission Tracking Sheet', 'Sheet20')

In [14]:
#create Name column
cohort_df['middle_name'] = cohort_df['Middle Name'].fillna("")
cohort_df['learner_name'] = cohort_df['First Name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['Surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['Name'] = cohort_df['learner_fullname'].str.title()


In [15]:
#set moderated poes
moderation_boolean = cohort_df['Selected for Moderation'] == 'Y'
cohort_df.loc[moderation_boolean, 'parent_folder_id'] = '12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B'

In [16]:
#set JHB poes
jhb_boolean = cohort_df['Campus'] == 'Johannesburg'
cohort_df.loc[~moderation_boolean & jhb_boolean, 'parent_folder_id'] = '1blnq78S8plz7PvPqoC4eiAG7eGtnAsGS'

In [17]:
#set CPT poes
cpt_boolean = cohort_df['Campus'] == 'Cape Town'
cohort_df.loc[~moderation_boolean & cpt_boolean, 'parent_folder_id'] = '18PR9IRA2WqkvRt-H0kzS0aiBDL92mbcV'

In [18]:
#set DBN poes
dbn_boolean = cohort_df['Campus'] == 'Durban'
cohort_df.loc[~moderation_boolean & dbn_boolean, 'parent_folder_id'] = '1IsEPiN8rBrYNLaPVyQ40lKByU1LZWhYS'

In [19]:
#subset useful columns
cohort_df = cohort_df[['Name', 'parent_folder_id']]

In [20]:
#merge cohort_df and folders_df
cohort_df = pd.merge(cohort_df, folders_df, how="left")

len(cohort_df)

217

**Create Local Library**

In [21]:
def create_poe_folder(poe_parent_folder_id, poe_folder_title):
    """
    Creates the student POE folder.
    
    Args:
        poe_parent_folder_id: A string representing the id of the poe parent folder
        poe_folder_title: A string representing the students fullname.
    """
    
    #declare global variables
    global poe_folder
    
    poe_folder = create_folder(poe_parent_folder_id, poe_folder_title)
    
    
def create_documents_folder(poe_documents_folder_id, evidence_documents_folder_id):
    """
    Creates document folder and copies documents files to folder.
    
    Args:
        poe_documents_folder_id: A string representing the id of the old poe documents folder.
        evidence_documents_folder_id A string representing the id of the old evidence folder.
    """
    
    #declare global variables
    global documents_folder, evidence_folders
    
    #create documents folder
    documents_folder = create_folder(poe_folder, 'POE Documents')
    
    #move all poe documents into documents folder
    move_file(poe_documents_folder_id, documents_folder)
    
    #move all loose documents from evidence folder to documents folder
    exclusion_array = ['documents', 'exams', 'Post Assessment Documents', 'technical reports', 'video demonstrations', 'video presentations']
    move_file(evidence_documents_folder_id, documents_folder, exclusion_array)
    
    #move all personal documents into documents folder
    evidence_folders = list_files(evidence_documents_folder_id)
    for folder in evidence_folders:
        if folder['title'] == 'documents':
            move_file(folder['id'], documents_folder)
            
            
def create_presentation_folder():
    """
    Creates presentation folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global presentation_folder, presentation_instruments_folder, presentation_evidence_folder
    
    #create presentation folder
    presentation_folder = create_folder(poe_folder, 'Module 1 - Presentation')
    
    #add presentation instruments folder
    presentation_instruments_folder = create_folder(presentation_folder, 'instruments')
    
    #add presentation evidence folder
    presentation_evidence_folder = create_folder(presentation_folder, 'evidence')
    
    #move all presentation evidence into presentation evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'video presentations':
            move_file(folder['id'], presentation_evidence_folder)
            
    #move all presentation post assessments docs into presentation folder
    exclusion_array = ['demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], presentation_folder,exclusion_array)
            
            
def create_demonstration_folder():
    """
    Creates demonstration folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global demonstration_folder, demonstration_instruments_folder, demonstration_evidence_folder
    
    #create demonstration folder
    demonstration_folder = create_folder(poe_folder, 'Module 2 - Demonstration')
    
    #add demonstration instruments folder
    demonstration_instruments_folder = create_folder(demonstration_folder, 'instruments')
    
    #add demonstration evidence folder
    demonstration_evidence_folder = create_folder(demonstration_folder, 'evidence')
    
    #move all demonstration evidence into demonstration evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'video demonstrations':
            move_file(folder['id'], demonstration_evidence_folder)
            
    #move all demonstration post assessments docs into demonstration folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], demonstration_folder,exclusion_array)
            
            
def create_reports_folder():
    """
    Creates reports folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global reports_folder, reports_instruments_folder, reports_evidence_folder
    
    #create reports folder
    reports_folder = create_folder(poe_folder, 'Module 3 - Reports')
    
    #add reports instruments folder
    reports_instruments_folder = create_folder(reports_folder, 'instruments')
    
    #add reports evidence folder
    reports_evidence_folder = create_folder(reports_folder, 'evidence')
    
    #move all report evidence into report evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'technical reports':
            move_file(folder['id'], reports_evidence_folder)
    
    #move all reports post assessments docs into reports folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], reports_folder,exclusion_array)
            
            
def create_exams_folder():
    """
    Creates exam folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global exams_folder, exams_formatives_folder, exams_summatives_folder
    
    #create exams folder
    exams_folder = create_folder(poe_folder, 'Module 4 - Exams')
    
    #add exams formative folder
    exams_formatives_folder = create_folder(exams_folder, 'formatives')
    
    #add exams summative folder
    exams_summatives_folder = create_folder(exams_folder, 'summatives')
    
    #move all exam evidence into exam evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'exams':
            exams_folders = list_files(folder['id'])
            for exam_folder in exams_folders:
                if exam_folder['title'] == 'formative':
                    move_file(exam_folder['id'], exams_formatives_folder)
                if exam_folder['title'] == 'summative':
                    move_file(exam_folder['id'], exams_summatives_folder)
                    
    #move all exams post assessments docs into exams folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], exams_folder,exclusion_array)
    

In [22]:
cohort_df.head()

,Name,parent_folder_id,Documents ID,Evidence ID
0,Adam Saul David Becker,12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B,19oj4tcbwqgRsGEAia15LRAyVKROwUE_F,16dCBkcvMqP5xURfMNBSRZIb98jIyPNrP
1,Abdur-Raheem Lee,1IsEPiN8rBrYNLaPVyQ40lKByU1LZWhYS,1yiXTWtlTyfI1Gz8LuMGlZLjLdaldZ0VK,1Sq-3cc2snBThL_DbPInn5N0TjUoGVf-p
2,Abdullah Naseem Ebrahim,18PR9IRA2WqkvRt-H0kzS0aiBDL92mbcV,1d347eyZe6W1BZwdQ2IEuyH2uuOZ2FCvu,1Uy1KpkoNrYIRbgi5bCoQAJJrkv54zUwT
3,Antonio Joseph Gioio,18PR9IRA2WqkvRt-H0kzS0aiBDL92mbcV,1oqFH2VgYoS_LDXtAroSku0M00wowu3j-,1uwlsATBghcgB5ysm8hYhumFn9RtMbT39
4,Abigail Hlalele,12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B,16lrp_FuEjwxzQ9nJyBONlZBxo__J9s6x,1s4a2_SCzt0Cx8iiOZfldLaJ5T-7ciaZY


**Main Function**

In [58]:


for index, row in cohort_df[cohort_df['Compiled'] != 'Yes'].iterrows():
    
    #set student values
    poe_parent_folder_id = row['parent_folder_id']
    poe_folder_title = row['Name']
    poe_documents_folder_id = row['Documents ID']
    evidence_documents_folder_id = row['Evidence ID']
    
    #log beginning
    print(f'{poe_folder_title}: Compilation Starting')
    
    #declare global variables
    poe_folder = ''
    documents_folder = ''
    evidence_folders = []
    presentation_folder = ''
    presentation_instruments_folder = ''
    presentation_evidence_folder = ''
    demonstration_folder = ''
    demonstration_instruments_folder = ''
    demonstration_evidence_folder = ''
    reports_folder = ''
    reports_instruments_folder = ''
    reports_evidence_folder = ''
    exams_folder = ''
    exams_formatives_folder = ''
    exams_summatives_folder = ''
    
    #execute main functions
    create_poe_folder(poe_parent_folder_id, poe_folder_title)
    create_documents_folder(poe_documents_folder_id, evidence_documents_folder_id)
    create_presentation_folder()
    create_demonstration_folder()
    create_reports_folder()
    create_exams_folder()
    
    #create new array
    array_df.append([poe_folder_title, poe_folder, documents_folder,
                presentation_folder, presentation_instruments_folder, presentation_evidence_folder,
                demonstration_folder, demonstration_instruments_folder, demonstration_evidence_folder,
                reports_folder, reports_instruments_folder, reports_evidence_folder,
                exams_folder, exams_formatives_folder, exams_summatives_folder])
    
    #log success
    cohort_df.loc[cohort_df['Name'] == row['Name'], 'Compiled'] = 'Yes'
    print(f'{poe_folder_title}: Successfuly compiled')

Damian Viljoen: Compilation Starting
Damian Viljoen: Successfuly compiled
Sebaratlane Musa Mashaba: Compilation Starting
Sebaratlane Musa Mashaba: Successfuly compiled


**Closing Function**

In [59]:
poe_df = pd.DataFrame(array_df, columns=['poe_folder_title', 'poe_folder', 'documents_folder',
                'presentation_folder', 'presentation_instruments_folder', 'presentation_evidence_folder',
                'demonstration_folder', 'demonstration_instruments_folder', 'demonstration_evidence_folder',
                'reports_folder', 'reports_instruments_folder', 'reports_evidence_folder',
                'exams_folder', 'exams_formatives_folder', 'exams_summatives_folder'])

**Export to Spreadsheet**

In [60]:
export_worksheet('POE Folders Compilation','Cohort',cohort_df)

In [61]:
export_worksheet('POE Folders Compilation','Folders', poe_df)